In [1]:
import pandas as pd
from pathlib import Path
from nltk.corpus import stopwords
from hmmlearn.hmm import MultinomialHMM
import nltk
import string
import re
import seaborn as sns
import gc
import numpy as np
from scipy.sparse import csr_matrix

from seqlearn.evaluation import bio_f_score
from seqlearn.hmm import MultinomialHMM
from seqlearn.evaluation import SequenceKFold

sns.set_context('talk', rc={'figure.figsize': (22, 18)})
sns.set_style('darkgrid')

%matplotlib inline 

In [2]:
DATA = Path('../data')
PROCESSED = DATA / 'processed_voa.tsv'
PUNCTUATION = list(string.punctuation)
TARGET = 'iob_ner'

In [3]:
def word_shape(word):
    t1 = re.sub('[A-Z]', 'X', word)
    t2 = re.sub('[a-z]', 'x', t1)
    return re.sub('[0-9]', 'd', t2)

def stem(word, stemmer):
    return stemmer.stem(word)

In [4]:
def get_X_y_lengths(df: pd.DataFrame, cols_to_keep=None, sequence_column='seq', target=TARGET, one_hot=False):
    if isinstance(sequence_column, str):
        sequence_column = [sequence_column]
    if cols_to_keep is None:
        cols_to_keep = {}
        
    y = df[target].cat.codes.values.copy()
    lengths = df.groupby(sequence_column, sort=False).count().iloc[:, 0].values
    if target in cols_to_keep:
        cols_to_keep.remove(target)
    cols_to_drop = set(df.columns) - cols_to_keep
    X = df.drop(cols_to_drop, axis=1)
    if one_hot:
        X = pd.get_dummies(X, dtype=np.bool, sparse=True)
    else:
        X = X.values if X.shape[1] > 1 else np.ravel(X.values)

    return X, y, lengths

In [5]:
def get_cv(X: np.ndarray, y: np.ndarray, lengths=None, n_folds=5):
    if lengths is None:
        lengths = X.shape[0]
    kf = SequenceKFold(lengths=lengths, n_folds=n_folds)
    for (train_ind, train_lengths, test_ind, test_lengths) in kf:
        yield X[train_ind], train_lengths, y[train_ind], X[test_ind], test_lengths, y[test_ind]

# Basic processing

In [ ]:
df = pd.read_csv(PROCESSED, sep='\t')
df.sample(5)

In [ ]:
df_without_punctuation = df[~df.token.isin(PUNCTUATION)].copy()

In [ ]:
stopwords_index = ~df_without_punctuation.lemma.str.lower().isin(stopwords.words('english'))
df_without_stopwords = df_without_punctuation[stopwords_index].copy()
len(df_without_stopwords)

In [ ]:
df_without_stopwords['shape'] = df.token.map(word_shape)

In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')
df_without_stopwords['stem'] = df_without_stopwords.lemma.map(lambda w: stem(w, stemmer))

In [ ]:
df_without_stopwords.to_csv(DATA / 'basic_processing.csv', header=True, index=False)

# EDA

In [ ]:
g = sns.catplot(x='iob_ner', data=df_without_stopwords, kind='count', aspect=1.5, color='b')
g.set_xticklabels(rotation=45);

In [ ]:
g = sns.catplot(x='iob_ner', data=df_without_stopwords[df_without_stopwords.iob_ner != 'O'], 
                kind='count', aspect=1.5, color='b')
g.set_xticklabels(rotation=45);

# Baseline models

## HMM

In [6]:
train = pd.read_csv(DATA / 'basic_processing.csv')
train.sample()

,token,pos,lemma,iob_ner,part,document,sentence,shape,stem
159453,aborted,JJ,aborted,O,p21,d0408,3,xxxxxxx,abort


In [7]:
train['seq'] = train.groupby(['document', 'part']).grouper.group_info[0]
train = train.astype('category')

In [8]:
print(train.stem.unique().shape[0])
print(train.token.unique().shape[0])
print(train.lemma.unique().shape[0])

23879
34898
27073


In [17]:
X, y, lengths = get_X_y_lengths(train, cols_to_keep={'lemma'}, one_hot=True)

In [18]:
X_sparse = csr_matrix(X.to_coo())

In [19]:
cv = SequenceKFold(lengths=lengths, n_folds=7)
i = 1
scores = []
fscores = []
print('Fold\tAccuracy\tF-score')
for train_ind, train_len, test_ind, test_len in cv:
    hmm = MultinomialHMM()
    
    train_x = X_sparse[train_ind]
    train_y = y[train_ind]
    
    hmm.fit(train_x, y=train_y, lengths=train_len)
    
    test_x = X_sparse[test_ind]
    test_y = y[test_ind]
    score = hmm.score(test_x, test_y, test_len)

    pred = hmm.predict(test_x, test_len)
    str_true = np.asarray(pd.Categorical.from_codes(test_y, train[TARGET].cat.categories), dtype=str)
    str_pred = np.asarray(pd.Categorical.from_codes(pred, train[TARGET].cat.categories), dtype=str)

    fscore = bio_f_score(str_true, str_pred)
    fscores.append(fscore)

    print(f'{i}\t{score}\t{fscore}')
    scores.append(score)
    i += 1
print(f'CV accuracy: {np.mean(scores)}')

print(f'CV bio f-score: {np.mean(fscores)}')

Fold	Accuracy	F-score
1	0.8243959530303909	0.41918610389183136
2	0.8218326338793587	0.4123370305506908
3	0.821276513392166	0.4081000987816925
4	0.8224001702111198	0.41246398887380376
5	0.8261239732587726	0.41019417475728154
6	0.8271624039065899	0.42428173033270644
7	0.8288452982028515	0.43798173456830103
CV accuracy: 0.824576706554464
CV bio f-score: 0.417792123108044
